In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5
no_cuda = False
log_interval = 100

# Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [4]:
train_paths = glob('fastcampus/dataset/mnist_png/training/*/*.png')[:1000]
test_paths = glob('fastcampus/dataset/mnist_png/testing/*/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [5]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        image = Image.open(path).convert("L")
        label = int(path.split('\\')[-2])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False
)

In [7]:
for i, data in enumerate(train_loader):
    if i == 0:
        print(data[0].shape, data[1].shape)
        break

torch.Size([64, 1, 28, 28]) torch.Size([64])


# Optimization

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Training

## (Training 까지 동일과정 반복)

In [9]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

Train Epoch: 1 [0/1000 (0%)]	Loss: 2.308139

Test set: Average loss: 0.5606, Accuracy: 980/1000 (98%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.516722

Test set: Average loss: 0.1423, Accuracy: 980/1000 (98%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.069972

Test set: Average loss: 0.1125, Accuracy: 980/1000 (98%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.030837

Test set: Average loss: 0.1072, Accuracy: 980/1000 (98%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.018309

Test set: Average loss: 0.1067, Accuracy: 980/1000 (98%)



# Save Model

공식 홈페이지에서는 바로 다음 단계에서 배울 전체 모델을 학습하는 것보다는 이 방법을 추천  
나중에 다른 모델에 적용하거나 수정하는 등 여러가지 이유로 용이하기 때문

### 모델만 weight만 저장

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [10]:
save_path = 'model_weight.pt'

# save_path 지정

# model의 weight만 저장하기 위함.


In [29]:
torch.save(model.state_dict(), save_path)
# model.state_dict를 통해 저장하고, save_path에 일치하는 파일명 
#(model_weight.pt) 로 내 workspace 경로에 저장됩니다.

In [25]:
model.state_dict().keys()

# 각 layer 별 weight가 들어있음

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [26]:
model.state_dict()['conv1.weight']

tensor([[[[-1.5675e-01, -1.2712e-01, -1.0717e-01, -1.0754e-02,  7.6381e-02],
          [ 9.4860e-02, -9.9893e-02, -3.7203e-02, -1.5823e-01,  1.4869e-01],
          [ 1.9968e-01,  1.9236e-01,  7.7892e-02, -4.7331e-02, -9.0103e-03],
          [ 7.1775e-02, -1.7648e-01,  7.8558e-03, -1.6794e-01, -2.0489e-01],
          [-8.2112e-03, -6.7429e-02, -1.3166e-01,  2.8188e-02,  3.8765e-03]]],


        [[[ 2.8782e-02, -1.1406e-01, -1.9818e-01,  2.3067e-03, -8.3430e-02],
          [-6.9352e-02,  1.4541e-01,  3.0975e-02,  8.9983e-02, -5.7169e-02],
          [-1.6069e-01,  3.8244e-02, -1.3349e-01,  2.4405e-02, -1.5647e-01],
          [ 1.4911e-01, -1.3763e-01,  1.7369e-01,  1.4208e-01,  1.5199e-01],
          [ 1.1858e-01,  4.5595e-02, -9.3454e-03, -2.3039e-03, -7.9053e-02]]],


        [[[-1.8555e-01,  1.8094e-01,  9.0124e-03,  9.3164e-02,  6.3902e-03],
          [ 5.6096e-02, -1.2574e-01, -1.5837e-01,  1.5066e-01,  7.4492e-02],
          [ 9.7355e-02, -2.0191e-01,  9.6093e-03, -1.8996e-01, -2.50

In [28]:
model.state_dict()['conv1.weight'].shape

# conv1 층의 모양 확인

torch.Size([20, 1, 5, 5])

In [12]:
model = Net().to(device)
weight_dict = torch.load(save_path)

# model이 현 device에 저장된 save_path로부터 대입

In [13]:
weight_dict.keys()



odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [14]:
weight_dict['conv1.weight'].shape


torch.Size([20, 1, 5, 5])

In [15]:
model.load_state_dict(weight_dict)
model.eval()
# 위 과정을 통해 최종적으로 load 확정

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# Save Entire model

In [30]:
save_path = 'model.pt'
# 전체 모델을 저장하기위한 path

In [17]:
torch.save(model, save_path)
# 모델을 통째로 저장

C:\Users\jusik\Anaconda3\envs\pytorchO\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [18]:
model = torch.load(save_path)
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# Save, Load and Resuming Training

In [19]:
checkpoint_path = 'checkpoint.pt'

In [20]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, checkpoint_path)

# epoch / model weight/ optimizer state / loss 등등을 한번에 저장하여 checkpoint_path에 저장

In [21]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# optimizer 불러오기
checkpoint = torch.load(checkpoint_path)
# 체크포인트 load

In [22]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])

In [23]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [31]:
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [32]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [34]:
epoch

5

In [37]:
loss

tensor(0.0129, device='cuda:0', requires_grad=True)

In [36]:
model.train()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)